In [2]:
import os
import sys
import torch
import torch.backends.cudnn as cudnn
from tqdm import tqdm
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from os import path
import numpy as np

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from network.TorchUtils import TorchModel
from features_loader import FeaturesLoaderVal
from utils.utils_ import get_torch_device
from utils.utils_ import register_logger 

Using GPU: NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [2]:
pip install cjm-pytorch-utils


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install scikit-learn



[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
print(get_torch_device())

Using GPU: NVIDIA GeForce RTX 3050 Ti Laptop GPU
cuda


In [3]:
!nvidia-smi

Thu Jul  4 14:44:36 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.99                 Driver Version: 555.99         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   55C    P5             15W /   20W |     378MiB /   4096MiB |      8%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Definitions

In [3]:
features_path = r'F:\VAD\AnomalyDetection\features'  # path to features
annotation_path = r'F:\VAD\AnomalyDetection\AnomalyDetectionCVPR2018-Pytorch\Test_Annotation.txt' 
model_path = r'F:\VAD\AnomalyDetection\AnomalyDetectionCVPR2018-Pytorch\exps\c3d\models\epoch_80000.pt'  # set logging file

# enable cudnn tune
cudnn.benchmark = True
device = get_torch_device()  # will use GPU if available, CPU otherwise

Using GPU: NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [4]:
f = open(features_path, 'r')
# f1 = open(features_path, 'r')
print(f.read())


PermissionError: [Errno 13] Permission denied: 'F:\\VAD\\AnomalyDetection\\features'

# Load dataset and detection model

In [5]:
data_loader = FeaturesLoaderVal(features_path=features_path,
                                    annotation_path=annotation_path)
data_iter = torch.utils.data.DataLoader(data_loader,
                                            batch_size=1,
                                            shuffle=False,
                                            num_workers=0,  # 4, # change this part accordingly
                                            pin_memory=True)

model = TorchModel.load_model(model_path).to(device).eval()

# Predict scores

In [6]:
y_trues = torch.tensor([])
y_preds = torch.tensor([])

with torch.no_grad():
    for features, start_end_couples, lengths in tqdm(data_iter):
        # features is a batch where each item is a tensor of 32 4096D features
        features = features.to(device)
        outputs = model(features).squeeze(-1)  # (batch_size, 32)
        for vid_len, couples, output in zip(lengths, start_end_couples, outputs.cpu().numpy()):
            y_true = np.zeros(vid_len)
            y_pred = np.zeros(vid_len)

            segments_len = vid_len // 32
            for couple in couples:
                if couple[0] != -1:
                    y_true[couple[0]: couple[1]] = 1

            for i in range(32):
                segment_start_frame = i * segments_len
                segment_end_frame = (i + 1) * segments_len
                y_pred[segment_start_frame: segment_end_frame] = output[i]

            if y_trues is None:
                y_trues = y_true
                y_preds = y_pred
            else:
                y_trues = np.concatenate([y_trues, y_true])
                y_preds = np.concatenate([y_preds, y_pred])


100%|██████████| 290/290 [00:35<00:00,  8.22it/s]


## Calculate ROC-AUC and plot the graph

In [7]:
fpr, tpr, thresholds = roc_curve(y_true=y_trues, y_score=y_preds, pos_label=1)

plt.figure()
lw = 2
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")

os.makedirs('graphs', exist_ok=True)
plt.savefig(path.join('graphs', 'roc_auc.png'))
plt.close()
print('ROC curve (area = %0.2f)' % roc_auc)


ROC curve (area = 0.69)


In [22]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import os

# Compute ROC curve and ROC area
fpr, tpr, thresholds = roc_curve(y_true=y_trues, y_score=y_preds, pos_label=1)
roc_auc = auc(fpr, tpr)

# Plotting the ROC curve
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")

# Save the plot to the specified directory
os.makedirs('graphs', exist_ok=True)
plt.savefig(os.path.join('graphs', 'roc_auc.png'))
plt.close()

print('ROC curve (area = %0.2f)' % roc_auc)


ROC curve (area = 0.69)


In [16]:
import numpy as np
threshold = 0.5
y_preds_binary = (y_preds >= threshold).astype(int)


TP = np.sum((y_preds_binary == 1) & (y_trues == 1))
FN = np.sum((y_preds_binary == 0) & (y_trues == 1))

TPR = TP / (TP + FN)

print(f"True Positive Rate (TPR): {TPR}")


True Positive Rate (TPR): 0.31821529543132737


In [17]:
import numpy as np

threshold = 0.5
y_preds_binary = (y_preds >= threshold).astype(int)

FP = np.sum((y_preds_binary == 1) & (y_trues == 0))
TN = np.sum((y_preds_binary == 0) & (y_trues == 0))

FPR = FP / (FP + TN)

print(f"False Positive Rate (FPR): {FPR}")


False Positive Rate (FPR): 0.11420691963807844


In [18]:
import numpy as np

# Assume y_preds and y_trues are defined elsewhere in your code
threshold = 0.5
y_preds_binary = (y_preds >= threshold).astype(int)

FP = np.sum((y_preds_binary == 1) & (y_trues == 0))
TN = np.sum((y_preds_binary == 0) & (y_trues == 0))

TNR = TN / (TN + FP)

print(f"True Negative Rate (TNR): {TNR}")


True Negative Rate (TNR): 0.8857930803619215


In [19]:
import numpy as np

# Assume y_preds and y_trues are defined elsewhere in your code
threshold = 0.5
y_preds_binary = (y_preds >= threshold).astype(int)

FN = np.sum((y_preds_binary == 0) & (y_trues == 1))
TP = np.sum((y_preds_binary == 1) & (y_trues == 1))

FNR = FN / (FN + TP)

print(f"False Negative Rate (FNR): {FNR}")


False Negative Rate (FNR): 0.6817847045686727


In [26]:
import numpy as np

def calculate_metrics(y_true, y_score, thresholds):
    tpr_list = []
    fpr_list = []
    tnr_list = []
    fnr_list = []
    
    for threshold in thresholds:
        # Binarize predictions
        y_preds_binary = (y_score >= threshold).astype(int)
        
        # Calculate confusion matrix components
        TP = np.sum((y_preds_binary == 1) & (y_true == 1))
        FN = np.sum((y_preds_binary == 0) & (y_true == 1))
        TN = np.sum((y_preds_binary == 0) & (y_true == 0))
        FP = np.sum((y_preds_binary == 1) & (y_true == 0))
        
        # Calculate rates
        TPR = TP / (TP + FN) if (TP + FN) > 0 else 0
        FPR = FP / (FP + TN) if (FP + TN) > 0 else 0
        TNR = TN / (TN + FP) if (TN + FP) > 0 else 0
        FNR = FN / (FN + TP) if (FN + TP) > 0 else 0
        
        tpr_list.append(TPR)
        fpr_list.append(FPR)
        tnr_list.append(TNR)
        fnr_list.append(FNR)
    
    return np.array(tpr_list), np.array(fpr_list), np.array(tnr_list), np.array(fnr_list)

# Example usage
# Replace y_true and y_score with your actual data
y_true = np.random.randint(0, 2, 10000)  # Example true labels
y_score = np.random.rand(10000)  # Example predicted scores

# Define a range of thresholds
thresholds = np.linspace(0, 1, 100)

# Calculate metrics
tpr, fpr, tnr, fnr = calculate_metrics(y_true, y_score, thresholds)

# Print an example result
print(f"True Positive Rate (TPR) at threshold 0.5: {tpr[np.searchsorted(thresholds, 0.5)]}")


True Positive Rate (TPR) at threshold 0.5: 0.4936025589764094


In [27]:
import numpy as np

def calculate_metrics(y_true, y_score, thresholds):
    tpr_list = []
    fpr_list = []
    tnr_list = []
    fnr_list = []
    
    for threshold in thresholds:
        # Binarize predictions
        y_preds_binary = (y_score >= threshold).astype(int)
        
        # Calculate confusion matrix components
        TP = np.sum((y_preds_binary == 1) & (y_true == 1))
        FN = np.sum((y_preds_binary == 0) & (y_true == 1))
        TN = np.sum((y_preds_binary == 0) & (y_true == 0))
        FP = np.sum((y_preds_binary == 1) & (y_true == 0))
        
        # Calculate rates
        TPR = TP / (TP + FN) if (TP + FN) > 0 else 0
        FPR = FP / (FP + TN) if (FP + TN) > 0 else 0
        TNR = TN / (TN + FP) if (TN + FP) > 0 else 0
        FNR = FN / (FN + TP) if (FN + TP) > 0 else 0
        
        tpr_list.append(TPR)
        fpr_list.append(FPR)
        tnr_list.append(TNR)
        fnr_list.append(FNR)
    
    return np.array(tpr_list), np.array(fpr_list), np.array(tnr_list), np.array(fnr_list)

# Example usage
# Replace y_true and y_score with your actual data
y_true = np.random.randint(0, 2, 10000)  # Example true labels
y_score = np.random.rand(10000)  # Example predicted scores

# Define a range of thresholds
thresholds = np.linspace(0, 1, 100)

# Calculate metrics
tpr, fpr, tnr, fnr = calculate_metrics(y_true, y_score, thresholds)

# Print an example result
print(f"False Positive Rate (FPR) at threshold 0.5: {fpr[np.searchsorted(thresholds, 0.5)]}")


False Positive Rate (FPR) at threshold 0.5: 0.4990885152926879


In [28]:
import numpy as np

def calculate_metrics(y_true, y_score, thresholds):
    tpr_list = []
    fpr_list = []
    tnr_list = []
    fnr_list = []
    
    for threshold in thresholds:
        # Binarize predictions
        y_preds_binary = (y_score >= threshold).astype(int)
        
        # Calculate confusion matrix components
        TP = np.sum((y_preds_binary == 1) & (y_true == 1))
        FN = np.sum((y_preds_binary == 0) & (y_true == 1))
        TN = np.sum((y_preds_binary == 0) & (y_true == 0))
        FP = np.sum((y_preds_binary == 1) & (y_true == 0))
        
        # Calculate rates
        TPR = TP / (TP + FN) if (TP + FN) > 0 else 0
        FPR = FP / (FP + TN) if (FP + TN) > 0 else 0
        TNR = TN / (TN + FP) if (TN + FP) > 0 else 0
        FNR = FN / (FN + TP) if (FN + TP) > 0 else 0
        
        tpr_list.append(TPR)
        fpr_list.append(FPR)
        tnr_list.append(TNR)
        fnr_list.append(FNR)
    
    return np.array(tpr_list), np.array(fpr_list), np.array(tnr_list), np.array(fnr_list)

# Example usage
# Replace y_true and y_score with your actual data
y_true = np.random.randint(0, 2, 10000)  # Example true labels
y_score = np.random.rand(10000)  # Example predicted scores

# Define a range of thresholds
thresholds = np.linspace(0, 1, 100)

# Calculate metrics
tpr, fpr, tnr, fnr = calculate_metrics(y_true, y_score, thresholds)

# Print an example result
print(f"True Negative Rate (TNR) at threshold 0.5: {tnr[np.searchsorted(thresholds, 0.5)]}")


True Negative Rate (TNR) at threshold 0.5: 0.4989010989010989


In [29]:
import numpy as np

def calculate_metrics(y_true, y_score, thresholds):
    tpr_list = []
    fpr_list = []
    tnr_list = []
    fnr_list = []
    
    for threshold in thresholds:
        # Binarize predictions
        y_preds_binary = (y_score >= threshold).astype(int)
        
        # Calculate confusion matrix components
        TP = np.sum((y_preds_binary == 1) & (y_true == 1))
        FN = np.sum((y_preds_binary == 0) & (y_true == 1))
        TN = np.sum((y_preds_binary == 0) & (y_true == 0))
        FP = np.sum((y_preds_binary == 1) & (y_true == 0))
        
        # Calculate rates
        TPR = TP / (TP + FN) if (TP + FN) > 0 else 0
        FPR = FP / (FP + TN) if (FP + TN) > 0 else 0
        TNR = TN / (TN + FP) if (TN + FP) > 0 else 0
        FNR = FN / (FN + TP) if (FN + TP) > 0 else 0
        
        tpr_list.append(TPR)
        fpr_list.append(FPR)
        tnr_list.append(TNR)
        fnr_list.append(FNR)
    
    return np.array(tpr_list), np.array(fpr_list), np.array(tnr_list), np.array(fnr_list)

# Example usage
# Replace y_true and y_score with your actual data
y_true = np.random.randint(0, 2, 10000)  # Example true labels
y_score = np.random.rand(10000)  # Example predicted scores

# Define a range of thresholds
thresholds = np.linspace(0, 1, 100)

# Calculate metrics
tpr, fpr, tnr, fnr = calculate_metrics(y_true, y_score, thresholds)

# Print an example result
print(f"False Negative Rate (FNR) at threshold 0.5: {fnr[np.searchsorted(thresholds, 0.5)]}")


False Negative Rate (FNR) at threshold 0.5: 0.5032692688725976
